In [11]:
from nlp import nlp
from morph import inflect

class Word:
    @staticmethod
    def create(token) -> 'Word':
        if token.upos == "PRON":
            return Pronoun(token)

        if token.upos == "DET":
            return Determiner(token)

        if token.upos == "VERB" or token.upos == "AUX":
            return Verb(token)

        return Word(token)

    def __init__(self, token):
        self._token = token

    def person_changed_text(self):
        return self._token.text


class Verb(Word):
    def __init__(self, token):
        self._token = token

    def person_changed_text(self):
        if 'VerbForm=Inf' in self._token.feats:
            return self._token.text

        person = '2per' if 'Person=1' in self._token.feats else '1per'
        inflected, _ = inflect(self._token.text, (frozenset({ person }), ))

        return inflected


class Pronoun(Word):
    _change_map = {
        'я': 'ты',
        'меня': 'тебя',
        'мне': 'тебе',
        'мной': 'тобой',

        'мы': 'вы',
        'нас': 'вас',
        'нам': 'вам',
        'нами': 'вами',

        'мой': 'твой',
        'моего': 'твоего',
        'моему': 'твоему',
        'моим': 'твоим',
        'моём': 'твоём',
        'моем': 'твоем',

        'моя': 'твоя',
        'моей': 'твоей',
        'мою': 'твою',

        'мое': 'твое',
        'моё': 'твоё',

        'мои': 'твои',
        'моих': 'твоих',
        'моим': 'твоим',
        'моими': 'твоими',

        'наш': 'ваш',
        'нашего': 'вашего',
        'нашему': 'вашему',
        'нашими': 'вашими',
        'наших': 'ваших',
    }

    _change_map.update({v: k for k, v in _change_map.items()})

    def person_changed_text(self):
        text = self._token.text.lower()

        print('key', text)
        print('found', Pronoun._change_map.get(text))

        return Pronoun._change_map.get(text, text)


class Determiner(Pronoun):
    """
    https://universaldependencies.org/ru/pos/DET.html
    Examples
        possessive determiners: мой, твой, его, её, наш, ваш, их  “my, your, his, her, our, your, their”
        reflexive possessive determiner: свой  “one’s own”
        demonstrative determiners: этот  as in Я видела эту машину вчера.  “I saw this car yesterday.”
        interrogative determiners: какой  as in Какая машина тебе нравится?  “Which car do you like?”
        relative determiners: который  as in Мне интересно, которая машина тебе нравится.  “I wonder which car you like.”
        relative possessive determiner: чей  “whose”
        indefinite determiners: некоторый
        total determiners: каждый
        negative determiners: никакой  as in У нас не осталось никаких машин.  “We have no cars available.”
    """
    pass

words = [Word.create(word) for sent in nlp('Ты мой котик. Я тебя люблю и ты будешь любить.').sentences for word in sent.words]

[w.person_changed_text() for w in words]

key ты
found я
key мой
found твой
key я
found ты
key тебя
found меня
key ты
found я


['я',
 'твой',
 'котик',
 '.',
 'ты',
 'меня',
 'любишь',
 'и',
 'я',
 'буду',
 'любить',
 '.']